# Document Data Extraction with GPT-4o and Evaluation using Prompt Flow

This notebook demonstrates how to use Prompt Flow in Azure AI Studio to extract structured JSON data from documents converted to images using GPT-4o in Azure OpenAI. The extracted data is then evaluated using an evaluation Prompt Flow to ensure the data is accurate, comparing it to ground truth data.

## Pre-requisites

The notebook uses Bash and [Azure CLI](https://learn.microsoft.com/cli/azure/install-azure-cli) to deploy all necessary Azure resources. Bash is available by default on Linux and macOS, and on Windows using [Windows Subsystem for Linux](https://docs.microsoft.com/windows/wsl/install). Alternatively, the repository includes a [devcontainer](./.devcontainer) configuration for Visual Studio Code, which includes a Linux environment with the necessary tools pre-installed.

Before continuing, ensure that you have selected the **Bash** kernel for the notebook. This can be found by clicking on the kernel selector in the top right corner of the notebook, selecting **Jupyter Kernel...** and then **Bash**.

![Select Bash kernel](./images/bash-kernel.png)

Running this notebook will deploy the following resources in your Azure subscription:
- Resource Group
- Managed Identity, with the following scoped role assignments:
  - Contributor (Resource Group) 
  - Storage Account Contributor (Storage Account)
  - Storage Blob Data Contributor (Storage Account)
  - Storage File Data Privileged Contributor (Storage Account)
  - Storage Table Data Contributor (Storage Account)
  - Key Vault Administrator (Key Vault)
  - ACR Pull (Container Registry)
  - ACR Push (Container Registry)
  - Cognitive Services Contributor (AI Services)
  - Cognitive Services OpenAI Contributor (AI Services)
  - Azure ML Data Scientist (AI Hub/Project)
- Storage Account, with a `document` container for storing the document images
- Key Vault
- Log Analytics Workspace, with an Application Insights instance
- Container Registry
- AI Services, with GPT-4o global standard model deployment (10K TPM)
- AI Hub & Project

These resources are deployed in a secure manner, with API key access disabled by default. Authentication and authorization are handled using Azure Role-Based Access Control (RBAC) and Managed Identity.

> **IMPORTANT:** Running the sample's evaluation prompt flow for each test case defined later with GPT-4o accrues token-based charges as would be expected running this in application code. Images are converted into tokens by converting your high resolution images into separate 512px tiled images. For more information, see the [Azure OpenAI image token overview](https://learn.microsoft.com/en-us/azure/ai-services/openai/overview#image-tokens-gpt-4-turbo-with-vision).

## Deploy the environment with Azure CLI, Bicep, and Prompt Flow CLI

The following will prompt you to login to your Azure account. Once logged in, the default subscription will be set, and the environment resources will be deployed.

> **Note**: If you have multiple subscription, you can change the default subscription by running `az account set --subscription <subscription-id>`.

The infrastructure deployment occurs at the subscription level, managing a resource group for you. The location of the deployment is set to **Sweden Central**, and this can be changed to another location that supports the GPT-4o model as a global standard deployment option. See the [`./infra/main.bicep`](./infra/main.bicep) file for more details.

> **Note**: Your user identity ID will be retrieved during the deployment and used to provide the necessary role assignments to your account.

Once the infrastructure deployment is complete, the Prompt Flow scenarios will be created in the AI Studio Project. The scenarios will be used to extract structured JSON data from documents converted to images using GPT-4o in Azure OpenAI and evaluate the extracted data.

### Understanding the deployment

#### Managed Identity

A [user-assigned Managed Identity](https://learn.microsoft.com/en-us/entra/identity/managed-identities-azure-resources/overview) is created for authenticating the Azure AI Hub and Projects with other Azure resources, including AI Services, Storage, and Key Vault, instead of using API keys.

Read more about [how to configure Azure OpenAI Service with managed identities](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/managed-identity).

#### Storage Account

A [Storage Account](https://learn.microsoft.com/en-us/azure/storage/common/storage-account-overview) is created to provide a data store for the Azure AI Hub and Project workspaces. Additionally, for this scenario, it is used to store the document images for processing.

#### Key Vault

A [Key Vault](https://learn.microsoft.com/en-us/azure/key-vault/general/basic-concepts) is created to store secrets, keys, and certificates used by the Azure AI Hub and Project workspaces. This resource is required, but not used in this scenario.

#### Log Analytics Workspace

A [Log Analytics Workspace](https://learn.microsoft.com/en-us/azure/azure-monitor/logs/data-platform-logs) is created to provide a centralized location for storing and analyzing logs from the various connected Azure resources.

#### Container Registry

A [Container Registry](https://learn.microsoft.com/en-us/azure/container-registry/container-registry-intro) is created to store and manage container images required for AI models used by the Azure AI Hub and Project workspaces. This resource is necessary for ideal AI environment setup, but not used in this scenario.

#### AI Services

An [AI Services](https://learn.microsoft.com/en-us/azure/ai-services/what-are-ai-services) resource is created to provide access to the GPT-4o model for the Azure AI Hub and Project workspaces. This resource is provided as a connection to the AI Hub and Project to be used as a reference in Prompt Flow scenarios.

#### AI Hub & Project

An [AI Studio Hub & Project](https://learn.microsoft.com/en-us/azure/ai-studio/what-is-ai-studio) is created to provide the necessary workspaces for building AI solution on Azure. These resources are used to create and manage Prompt Flow scenarios for document data extraction and evaluation.

#### Prompt Flow Scenarios

Two Prompt Flow scenarios are created in the AI Studio Project. The [Data Extraction Prompt Flow](./promptflow/document-data-extraction/flow.dag.yaml) extracts structured JSON data from documents converted to images using GPT-4o in Azure OpenAI. The extracted data is then evaluated using a [Data Extraction Evaluation Prompt Flow](./promptflow/document-data-extraction-evaluation/flow.dag.yaml) to ensure the data is accurate, comparing it to ground truth data.

In [ ]:
# Set necessary Azure infrastructure deployment variables
deploymentName='doc-extract-eval-pf'
location='swedencentral'

In [ ]:
# Set the Azure deployment subscription to the default subscription (or set it manually)
subscriptionId=$(az account list --query "[?isDefault].id" -o tsv)
# subscriptionId='00000000-0000-0000-0000-000000000000' # Uncomment this line to set the subscription manually

az account set --subscription $subscriptionId
echo "Subscription set to $subscriptionId"

In [ ]:
# Run the environment setup script to deploy the Bicep template (./infra/deploy.sh), and deploy the Prompt Flow scenarios (./promptflow/deploy.sh)
deploymentOutputs=$(./setup-environment.sh --deploymentName $deploymentName --location $location --skipInfrastructure)

## Running the Data Extraction and Evaluation Prompt Flow scenarios in the Azure AI Studio



### Understand the Data Extraction Prompt Flow scenario

![image.png](./images/data-extraction-flow.png)

This Prompt Flow is the core logic for loading images of documents from the Azure Storage Account into the correct format for the Azure OpenAI request, and then extracting structured JSON data from the document images using the GPT-4o model and prompts.

This flow is considered a ["standard" flow](https://microsoft.github.io/promptflow/concepts/concept-flows.html#flow-types), which represents application logic that can be deployed as a standalone endpoint for a larger AI solution.

The flow consists of the following inputs:

- **system_prompt** - The default instruction prompt that tailors the model to a specific task, in this example, document data extraction to structured JSON objects.
- **extraction_prompt** - The specific scenario instruction for performing the extraction containing key rules for performing an extraction on a type of document, including the expected output JSON object schema.
- **storage_account_name** - The name of an Azure Storage Account where document images are stored that the compute's managed identity can access with appropriate read RBAC.
- **blob_container_name** - The name of the blob container in the Azure Storage Account where the document images are stored.
- **temperature** - The creativity or randomness control. A lower number will result in more deterministic and focused outputs, great for accurate data extraction.
- **top_p** - The generated token probability control. A lower number will result in the model only consider this top % probability mass for the next token, resulting in more deterministic outputs.

> Note: In the deployed scenario, default values are provided using the deployed infrastructure. These can be run for your validation purposes of the Prompt Flow Python logic. When the evaluation flow runs a batch of tests later, these values will be overridden with the specific test values.

The steps in the flow are as follows:

1. **load_images** - The flow will create a `BlobServiceClient` to retrieve the list of blobs in the specified container and download them using the managed identity. Once downloaded, they are converted to a base64 encoded URI for the Azure OpenAI request, and returned as an array for processing. It has the following input parameters:
   - **storage_account_name** - The name of the Azure Storage Account.
   - **image_container_name** - The name of the blob container in the Azure Storage Account where the document images are stored.
2. **extract_document_data** - Using the Azure OpenAI Python SDK, the flow will construct a message containing the system prompt, extraction prompt, and the base64 encoded URI for each of the document images. This message is then sent to the API for the deployed GPT-4o model for extraction. The response is then provided as an output, to be later used by the evaluation flow. It has the following input parameters:
   - **aoai_connection** - The Prompt Flow connection to an Azure OpenAI service.
   - **model_deployment** - The name of the GPT model to use for the extraction, in this case, GPT-4o.
   - **system_prompt** - The default instruction prompt that tailors the model to a specific task.
   - **extraction_prompt** - The specific scenario instruction for performing the extraction, including the expected output JSON object schema.
   - **temperature** - The creativity or randomness control.
   - **top_p** - The generated token probability control.
   - **image_uris** - An array of base64 encoded URIs for the document images, provided by the `load_images` step.

### Understand the Data Extraction Evaluation Prompt Flow scenario

![image.png](./images/data-extraction-eval-flow.png)

This flow is considered an ["evaluation" flow](https://microsoft.github.io/promptflow/concepts/concept-flows.html#flow-types), which enables it to be used as part of automated evaluations in the Azure AI Studio to run batch tests over a standard flow, such as the data extraction scenario.

Evaluation flows take a batch of tests as a JSON lines file, and runs each test through the standard flow by providing the input parameters from the test case. These test cases allow you to establish baseline performance metrics for your extraction prompts, enabling you to experiment and improve the accuracy of your document data extraction scenarios.

The [`data.jsonl` file](./tests/data.jsonl) provided demonstrates how to construct the test cases for the batch evaluation. Each test case consists of the required inputs for the standard flow, as well as the expected output JSON object. The sample test cases provided can be used later to evaluate the effectiveness of the data extraction prompt for the [`tests/Invoice.pdf`](./tests/Invoice.pdf) document.

In this sample, the structure for each test case is as follows:

```json
{
    "system_prompt": "",
    "extraction_prompt": "",
    "blob_container_name": "",
    "storage_account_name": "",
    "temperature": 0.1,
    "top_p": 0.1,
    "expected": {}
}
```

Please feel free to modify the `data.jsonl` file to include additional scenarios with a tweaked extraction prompt to test the accuracy of the data extraction.

> **Note:** The expected output JSON object must be a valid JSON object that represents the known output based on a human analysis of a document.

The flow consists of the following inputs:

- **expected** - The JSON object that represents the expected, known output based on a human analysis of a document, also known as the ground truth or golden data.
- **actual** - The JSON object output generated by the data extraction prompt flow using the GPT-4o model.

The steps in the flow are as follows:

1. **compare_results** - The flow uses the ground truth and actual JSON objects to compare each of the keys and values to determine the number of exact matches. For objects with nested objects or arrays, the flow will recursively compare each key and value to determine the number of exact matches. Once determined, the flow will output the number of exact matches as a percentage of the total number of keys and values in the ground truth JSON object. This provides a guide for evaluating the accuracy of the scenario being evaluated.

### Uploading the document images to the Azure Storage Account

With an understanding of the scenarios, the first step is to convert a document into images, and upload them to the Azure Storage Account. The document images will be used as input for the data extraction scenario.

For this scenario, we have provided a sample invoice document, [`tests/Invoice.pdf`](./tests/Invoice.pdf), which will be converted to images and uploaded to the Azure Storage Account.

To test your own documents, update the variables in the cell below with the appropriate path to your document, and update the [`data.jsonl`](./tests/data.jsonl) file with test cases that match your extraction scenarios by providing an appropriate `system_prompt`, `extraction_prompt`, and `expected` JSON object.

In [ ]:
# Variables for creating and uploading the document images
pdf_file_path='./tests/Invoice.pdf'
output_dir='./tests/Invoice/'

storageAccountName=$(echo $deploymentOutputs | jq -r '.infrastructureOutputs.storageAccountInfo.value.name')
documentImageContainerName=$(echo $deploymentOutputs | jq -r '.infrastructureOutputs.storageAccountInfo.value.documentImageContainerName')

echo "Storage account name: $storageAccountName"
echo "Document image container name: $documentImageContainerName"

In [ ]:
# Convert the PDF file to images
python3 ./scripts/pdf_to_image.py $pdf_file_path $output_dir

In [ ]:
# Clear the existing document images in the Azure Blob Storage container
az storage blob delete-batch --account-name $storageAccountName --source $documentImageContainerName --auth-mode login

In [ ]:
# Upload the images to the Azure storage account
echo "Uploading document images from $output_dir to the container $documentImageContainerName storage account $storageAccountName..." >&2
az storage blob upload-batch --account-name $storageAccountName --destination $documentImageContainerName --source $output_dir --overwrite --auth-mode login

### Open the Azure AI Studio

To continue with this sample, we will jump into the Azure AI Studio. This section will walk through using the Azure AI Studio portal UI to continue.

In [ ]:
# Open the AI Studio Hub project URL
resourceGroupName=$(echo $deploymentOutputs | jq -r '.infrastructureOutputs.resourceGroupInfo.value.name')
aiHubProjectName=$(echo $deploymentOutputs | jq -r '.infrastructureOutputs.aiHubProjectInfo.value.name')

url="https://ai.azure.com/projectflows?wsid=/subscriptions/$subscriptionId/resourcegroups/$resourceGroupName/providers/Microsoft.MachineLearningServices/workspaces/$aiHubProjectName"
open $url

echo "The AI Studio Hub project is available at: $url" >&2

### Upload the test data to the Azure AI Studio project

After launching the Azure AI Studio portal, follow these steps to upload the `data.jsonl` file to the project:

> **Note:** Before uploading the data, ensure that the `storage_account_name` property for each test case is set to the name of the Azure Storage Account created during the infrastructure deployment. You can find the value above in the **Uploading the document images to the Azure Storage Account** section.

1. Navigate to the **Components > Data** section in the main menu for the project.
2. Click on the **New data** button.
3. Select the **Upload files/folders** option for the Data source, and choose the `data.jsonl` file from the `tests` folder.
    > ![image.png](./images/upload-eval-data.png)
4. Click **Next** and provide a **Data name** for the file, such as **eval_data**.
    - We will use this data name later when configuring the evaluation flow batch run.

### Create the custom evaluation run

After uploading the test data, follow these steps to create a custom evaluation run for the data extraction evaluation scenario:

1. Navigate to the **Tools > Prompt flow** section in the main menu for the project.
2. Select the `data-extraction` flow from the list.
    - **Note:** This is the scenario flow that performs the extraction that we will evaluate using the custom data extraction evaluation flow.
3. In the top-right corner, click on the **Start compute session** button to initialize a serverless compute session for the evaluation.
    - **Note:** This may take a few minutes to start. Once started, the button will change to **Compute session running**.
4. Once the compute session is running, click on the **Evaluate > Custom evaluation** button.
5. From this dialog, progress to the **Batch run settings** and select the **eval_data** data source that was uploaded earlier. This will map the data to the input parameters of the extraction flow.
    > ![image.png](./images/batch-run-settings.png)
6. From the **Evaluation settings**, select the custom **data-extraction-eval** flow from the available options.
    > ![image.png](./images/select-evaluation.png)
7. Once selected, configure the evaluation flows inputs to map the **actual** value to the output of the extraction flow.
    > ![image.png](./images/configure-evaluation.png)
8. Finally, review and submit the evaluation run.

### Review the evaluation results

After submitting the evaluation run, the Azure AI Studio will process the batch of tests and provide a summary of the results. The evaluation flow will compare the actual output generated by the data extraction flow to the expected output provided in the test data.

You can view the results of the evaluation run by navigating to the **Tools > Evaluation** section in the main menu for the project.

![image.png](./images/evaluation-runs.png)

Clicking into the run, you can view the summary of each test case, including the expected and actual outputs, as well as the results that contains the valid and invalid keys, plus the overall percentage accuracy.

![image.png](./images/evaluation-run-summary.png)
![image.png](./images/evaluation-run-details.png)